In [ ]:
import stanza
import openai
import pandas as pd
import time
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from torch.nn.functional import softmax
from sklearn.metrics import accuracy_score, classification_report
import pandas as pd
import random

In [ ]:

# Sample positive feedback examples
positive_feedback = [
    "The product exceeded our expectations, and shipping was prompt.",
    "Support team was quick to resolve our concerns professionally.",
    "Pricing strategy is competitive, allowing better profit margins.",
    "Customer demand has been steadily increasing for this product.",
    "Stock availability has improved, making inventory management easier.",
    "The latest update enhanced functionality and user experience.",
    "Commission structure has been motivating our sales team effectively.",
    "The training program was well-structured and very informative.",
    "Timely assistance from support made issue resolution seamless.",
    "Secure packaging has minimized damage during product transit.",
    "The quality of materials used in the product is outstanding.",
    "Timely communication from the team has helped in business planning.",
    "Product performance is excellent, leading to repeat orders.",
    "Sales process has been streamlined with better technology support.",
    "We appreciate the transparency in pricing and contract terms.",
    "Marketing support has helped boost sales and brand recognition.",
    "Technical documentation provided was clear and very useful.",
    "Product availability has been consistent, preventing stockouts.",
    "The mobile app interface has made order management more efficient.",
    "Dedicated account manager support has been very helpful.",
    "After-sales service has been excellent, increasing customer retention.",
    "The onboarding process was smooth and well-organized.",
    "Frequent updates ensure the product stays ahead of competitors.",
    "Bulk order processing has been made faster and more efficient.",
    "The product warranty terms are favorable and build customer trust.",
    "Product durability has significantly reduced return rates.",
    "Co-branding opportunities have strengthened our market presence.",
    "Lead times for order fulfillment have improved remarkably.",
    "We have received great promotional support from the company.",
    "Collaboration has been seamless, with clear communication at all stages.",
    "The customer service team is knowledgeable and provides quick resolutions.",
    "User interface is intuitive, making navigation smooth and effortless.",
    "Frequent software updates have significantly enhanced performance.",
    "The product offers great value for money compared to competitors.",
    "Order processing time has been significantly reduced, improving efficiency.",
    "The loyalty program rewards have increased customer retention.",
    "The product's reliability has strengthened customer confidence.",
    "Customer feedback is actively considered for product improvements.",
    "The company maintains transparency in all transactions.",
    "Live chat support is highly responsive and effective.",
    "Product packaging is eco-friendly and sustainable.",
    "The new automation features have improved workflow productivity.",
    "Marketing campaigns have increased brand visibility and sales.",
    "The detailed user manual simplifies the learning process.",
    "The vendor proactively communicates about product enhancements.",
    "Technical support team resolves issues within a short turnaround time.",
    "The product is well-engineered, leading to high durability.",
    "Collaboration tools have improved team efficiency and project management.",
    "The seamless integration with third-party apps enhances usability.",
    "Security measures implemented have increased trust in the platform.",
    "The trial period allowed a thorough evaluation of product features.",
    "Customizable options make the product suitable for various business needs.",
    "Subscription plans are flexible, accommodating different budget levels.",
    "The analytics dashboard provides valuable insights for business growth.",
    "The onboarding guide made setting up the system quick and hassle-free.",
    "Multi-language support has broadened our customer base.",
    "Return and exchange policies are customer-friendly and efficient.",
    "The product roadmap aligns well with industry trends and demands.",
    "The sales team is proactive and offers personalized recommendations.",
    "Account management support is outstanding, ensuring a smooth experience.",
    "The discount structures have made bulk purchasing more attractive.",
    "Shipping speeds have consistently met or exceeded expectations.",
    "Customer testimonials reflect high satisfaction with the product.",
    "The mobile version of the platform is just as robust as the desktop one.",
    "Personalized marketing efforts have improved conversion rates.",
    "The payment process is smooth, with multiple options for convenience.",
    "The company's commitment to innovation keeps the product competitive.",
    "Training sessions are comprehensive and cater to different skill levels.",
    "Regular product webinars provide ongoing education and best practices.",
    "The scalability of the platform supports business growth effectively.",
    "Vendor relationships are well-managed, ensuring continuous supply.",
    "Promotional discounts have driven more customer engagement.",
    "Product stability has led to fewer service interruptions.",
    "AI-powered features have improved automation and decision-making.",
    "User feedback implementation has led to noticeable enhancements.",
    "The company is quick to adapt to market changes and user demands.",
    "Seamless cross-platform compatibility makes the product highly versatile.",
    "Customer appreciation programs have strengthened brand loyalty.",
    "The easy-to-use reporting tools help track performance effectively.",
    "Strong community support provides additional resources and best practices.",
    "The flexible pricing model makes it accessible to businesses of all sizes."
]

# Sample negative feedback examples
negative_feedback = [
    "The product quality has been inconsistent, causing customer complaints.",
    "Support response times are too slow, affecting our business.",
    "Pricing is too high compared to competitors, impacting sales.",
    "We are facing stock shortages frequently, disrupting sales.",
    "Logistics delays have led to missed deadlines for customers.",
    "The software has frequent bugs, causing operational issues.",
    "Incentives are not attractive enough to drive more sales.",
    "Training was too generic and did not cover specific use cases.",
    "Support team is difficult to reach during critical issues.",
    "Packaging is poor, leading to damaged products in transit.",
    "The user interface is outdated and difficult to navigate.",
    "Frequent downtime of the platform is causing major disruptions.",
    "Customer service representatives lack technical knowledge.",
    "Order processing time is too slow, leading to frustrated customers.",
    "Return and refund policies are unclear and complicated.",
    "The mobile app crashes frequently, making it unreliable.",
    "Discounts and promotions are not competitive enough.",
    "Product descriptions are misleading, causing dissatisfaction.",
    "The website search function does not work efficiently.",
    "Lack of multilingual support is limiting global reach.",
    "Products often arrive later than the estimated delivery date.",
    "The chatbot support is ineffective and gives irrelevant answers.",
    "Promotional emails are too frequent and feel like spam.",
    "Limited payment options make transactions inconvenient.",
    "The documentation lacks detailed troubleshooting steps.",
    "Some features advertised are missing or do not work as expected.",
    "Customer feedback is not being implemented into product updates.",
    "Account security measures are weak, raising privacy concerns.",
    "Shipping costs are too high compared to other vendors.",
    "The checkout process is confusing and leads to abandoned carts.",
    "Product inventory is not updated in real time, leading to order cancellations.",
    "Customer loyalty programs are not rewarding enough.",
    "The company does not handle complaints in a timely manner.",
    "Notifications are excessive and cannot be customized.",
    "Warranty claims take too long to process.",
    "The training materials provided are outdated.",
    "Difficulties in reaching an actual human for customer support.",
    "Lack of integration with third-party services is limiting usability.",
    "The product's lifespan is much shorter than expected.",
    "The response from the escalation team is often unhelpful.",
    "Key features require additional payments, making it expensive.",
    "The UI lacks customization options for different user needs.",
    "Pricing structures are confusing and not transparent.",
    "The software updates introduce more bugs than they fix.",
    "The onboarding process is cumbersome and not user-friendly.",
    "The FAQ section is not detailed enough to answer common questions.",
    "Long wait times for live chat support.",
    "The product is not compatible with industry-standard formats.",
    "Vendor communication is inconsistent, leading to uncertainty.",
    "Inconsistent quality control results in defective products.",
    "The system does not support bulk operations efficiently.",
    "No clear roadmap for upcoming feature improvements.",
    "Customization options are too limited for enterprise users.",
    "Order tracking information is often inaccurate.",
    "Shipping partners are unreliable, causing delays.",
    "No easy way to escalate urgent issues to management.",
    "Packaging is excessive, leading to environmental concerns.",
    "The help desk tickets take too long to be resolved.",
    "Customer support scripts feel robotic and impersonal.",
    "The service lacks proactive communication about known issues.",
    "The platform's navigation is unintuitive, making tasks take longer.",
    "System crashes unexpectedly, causing loss of progress and data.",
    "Support team lacks responsiveness, leading to unresolved issues.",
    "Product availability is inconsistent, leading to unfulfilled orders.",
    "Hidden fees in the billing process make pricing unclear.",
    "Order cancellations happen too frequently without explanation.",
    "The learning curve for new users is too steep.",
    "Customer inquiries often go unanswered for extended periods.",
    "Limited language options make the service inaccessible for many users.",
    "Mobile notifications are excessive and cannot be properly customized.",
    "Shipping partners are unreliable, frequently misplacing packages.",
    "Website speed is slow, making browsing frustrating.",
    "The refund process is complex and takes too long to complete.",
    "Live chat support is often unavailable during peak hours.",
    "Product updates remove useful features instead of improving them.",
    "Security concerns arise due to frequent account breaches.",
    "The software lacks automation features that competitors offer.",
    "Error messages are vague, making troubleshooting difficult.",
    "The knowledge base articles are outdated and lack relevant details.",
    "System maintenance happens at inconvenient times, disrupting usage.",
    "Customer complaints do not receive timely follow-ups.",
    "Limited inventory tracking results in overselling issues.",
    "Platform customizations are minimal, reducing flexibility for users.",
    "Multi-step authentication slows down the login process unnecessarily.",
    "Products often arrive with missing accessories or parts.",
    "The checkout process has unnecessary steps, leading to cart abandonment.",
    "The analytics dashboard lacks crucial insights needed for business decisions.",
    "Warranty support is hard to claim and lacks clear instructions.",
    "Competitor products offer better value for money.",
    "Subscription cancellation is overly complicated and unclear.",
    "Platform integration with third-party apps is unstable.",
    "Repeated outages make the service unreliable for business use.",
    "Customer requests for new features are ignored.",
    "The interface does not scale well on different screen sizes.",
    "Misleading marketing creates unrealistic expectations for the product.",
    "The onboarding tutorial does not cover key functions.",
    "Support representatives provide inconsistent solutions to the same issue.",
    "Trial periods are too short to evaluate the product properly.",
    "Search filters do not return accurate results.",
    "Frequent password reset requests indicate security flaws.",
    "The chatbots provide scripted responses that do not solve problems.",
    "Order tracking is delayed and does not reflect real-time status.",
    "User permissions are difficult to configure for different roles.",
    "System downtime during updates is not communicated in advance.",
    "The platform lacks offline functionality for critical operations.",
    "In-app advertisements interfere with user experience.",
    "Pricing tiers do not offer enough flexibility for small businesses.",
    "Support calls have long wait times with no call-back option.",
    "Video tutorials do not address common user concerns.",
    "Email support responses are generic and do not resolve the issue.",
    "The company lacks transparency in product development roadmaps."
]

# Generate 500 positive and 500 negative feedback samples
num_samples = 500

positive_samples = [{"Feedback_ID": i+1, "Feedback_Text": random.choice(positive_feedback), "Sentiment": "Positive"} for i in range(num_samples)]
negative_samples = [{"Feedback_ID": i+num_samples+1, "Feedback_Text": random.choice(negative_feedback), "Sentiment": "Negative"} for i in range(num_samples)]

# Combine both lists
dataset = positive_samples + negative_samples
random.shuffle(dataset)  # Shuffle to mix positive and negative feedback

# Create DataFrame
df = pd.DataFrame(dataset)

# Save as CSV file
csv_filename = "channel_partner_feedback.csv"
df.to_csv(csv_filename, index=False)

csv_filename


In [ ]:
%%writefile .env
OPENAI_API_KEY=your-openai-api-key-here


In [ ]:

# Load the .env file
load_dotenv()

# Get API key from .env
openai.api_key = os.getenv("OPENAI_API_KEY")

if not openai.api_key:
    raise ValueError("Error: OpenAI API key not found! Make sure the .env file is correctly loaded.")


In [ ]:
def get_gpt_sentiment(text):
    try:
        response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",
            request_timeout=10,
            messages=[
                {"role": "system", "content": "Classify this text strictly as Positive or Negative only."},
                {"role": "user", "content": text}
            ]
        )
        sentiment = response['choices'][0]['message']['content'].strip().lower()
        return "Positive" if "positive" in sentiment else "Negative"
    except Exception as e:
        print(f"Error: {e}")
        return "Negative"


In [ ]:

# Load Data
df = pd.read_csv("channel_partner_feedback.csv")  # Ensure it has 'text' and 'true_label' columns

# Analyze Sentiment for GPT (Add rate-limiting with sleep)
df["gpt_prediction"] = df["Feedback_Text"].apply(
    lambda x: (time.sleep(1), get_gpt_sentiment(x))[1]
)


# Print classification report
print("\n GPT Model Performance:")
print(classification_report(df["Sentiment"], df["gpt_prediction"]))


In [4]:

# Load Manually Labeled Dataset
df = pd.read_csv("channel_partner_feedback.csv")  # Ensure it has 'text' and 'true_label' columns

# Initialize Stanza NLP Model
stanza.download("en")  
nlp = stanza.Pipeline(lang="en", processors="tokenize,sentiment")

# Load RoBERTa Sentiment Analysis Model
roberta_model_name = "cardiffnlp/twitter-roberta-base-sentiment"
tokenizer = AutoTokenizer.from_pretrained(roberta_model_name)
roberta_model = AutoModelForSequenceClassification.from_pretrained(roberta_model_name)
roberta_model.eval()

# Function to Get Sentiment from Stanza
def get_stanza_sentiment(text):
    doc = nlp(text)
    sentiment_score = doc.sentences[0].sentiment  # (0 = Negative, 1 = Neutral, 2 = Positive)
    # Convert Stanza output to only Positive/Negative
    return "Positive" if sentiment_score == 2 else "Negative"


# Function to Get Sentiment from RoBERTa
def analyze_sentiment(text, threshold=0.5):
    """
    Analyze sentiment using RoBERTa and classify as Positive, or Negative.
    """
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=512)
    
    with torch.no_grad():
        outputs = roberta_model(**inputs)
        logits = outputs.logits

    probs = softmax(logits, dim=-1).numpy()[0]  # Convert logits to probabilities
    negative_prob, neutral_prob, positive_prob = probs  # Extract individual probabilities

    # Classify based on confidence threshold
    if positive_prob >= threshold:
        return "Positive"
    else:
        return "Negative"

# Store Predictions
roberta_predictions = []
stanza_predictions = []


# Loop Through Dataset and Get Predictions
for index, row in df.iterrows():
    roberta_pred =analyze_sentiment(row["Feedback_Text"])
    stanza_pred = get_stanza_sentiment(row["Feedback_Text"])

    roberta_predictions.append(roberta_pred)
    stanza_predictions.append(stanza_pred)

# Add Predictions to DataFrame
df["roberta_prediction"] = roberta_predictions
df["stanza_prediction"] = stanza_predictions


# Accuracy Calculation
print("\n Accuracy and Classification Reports:")

print("\n RoBERTa Model Performance:")
print(classification_report(df["Sentiment"], df["roberta_prediction"]))

print("\n Stanza Model Performance:")
print(classification_report(df["Sentiment"], df["stanza_prediction"]))


2025-04-04 19:18:19 INFO: Downloaded file to C:\Users\291688\stanza_resources\resources.json
2025-04-04 19:18:19 INFO: Downloading default packages for language: en (English) ...
2025-04-04 19:18:21 INFO: File exists: C:\Users\291688\stanza_resources\en\default.zip
2025-04-04 19:18:25 INFO: Finished downloading models and saved to C:\Users\291688\stanza_resources
2025-04-04 19:18:25 INFO: Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


2025-04-04 19:18:26 INFO: Downloaded file to C:\Users\291688\stanza_resources\resources.json
2025-04-04 19:18:26 WARNING: Language en package default expects mwt, which has been added
2025-04-04 19:18:26 INFO: Loading these models for language: en (English):
| Processor | Package        |
------------------------------
| tokenize  | combined       |
| mwt       | combined       |
| sentiment | sstplus_charlm |

2025-04-04 19:18:26 INFO: Using device: cpu
2025-04-04 19:18:26 INFO: Loading: tokenize
2025-04-04 19:18:26 INFO: Loading: mwt
2025-04-04 19:18:26 INFO: Loading: sentiment
2025-04-04 19:18:29 INFO: Done loading processors!



 Accuracy and Classification Reports:

 RoBERTa Model Performance:
              precision    recall  f1-score   support

    Negative       0.89      1.00      0.94       500
    Positive       1.00      0.88      0.93       500

    accuracy                           0.94      1000
   macro avg       0.94      0.94      0.94      1000
weighted avg       0.94      0.94      0.94      1000


 Stanza Model Performance:
              precision    recall  f1-score   support

    Negative       0.85      0.98      0.91       500
    Positive       0.97      0.83      0.89       500

    accuracy                           0.90      1000
   macro avg       0.91      0.90      0.90      1000
weighted avg       0.91      0.90      0.90      1000



In [ ]:
# Accuracy Calculation
print("\n Accuracy and Classification Reports:")

print("\n RoBERTa Model Performance:")
print(classification_report(df["Sentiment"], df["roberta_prediction"]))

print("\n Stanza Model Performance:")
print(classification_report(df["Sentiment"], df["stanza_prediction"]))

print("\n GPT Model Performance:")
print(classification_report(df["Sentiment"], df["gpt_prediction"]))
